# Metaprogramación en Python

Este proyecto demuestra cómo realizar una **regresión lineal simple** utilizando conceptos de metaprogramacion en Python. 

---

## Requisitos del Sistema

1. **Python 3.8 o superior**.
2. **Anaconda (opcional)**: Se recomienda instalar Anaconda para manejar el entorno con facilidad.
3. **Librerías requeridas**:
   - `numpy`
   - `scikit-learn`
   - `matplotlib`
   - `jupyter`

---

## Instalación

### Opción 1: Usando Anaconda (Recomendado)
1. Descarga e instala **Anaconda**: [https://www.anaconda.com/](https://www.anaconda.com/).
2. Crea un entorno para este proyecto:
   ```bash
   conda create -n mi_entorno python=3.9
   conda activate mi_entorno


In [97]:
import time
import numpy as np
import inspect
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from abc import ABC, abstractmethod


In [98]:
# Decorador para medir el tiempo de ejecución
def medir_tiempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        print(f"{func.__name__} ejecutado en {fin - inicio:.6f} segundos")
        return resultado
    return wrapper

In [99]:
# Metaclase que obliga a implementar 'train' y 'predict'
class ModeloMLMeta(type):
    def __new__(cls, name, bases, dct):
        if 'train' not in dct or 'predict' not in dct:
            raise TypeError(f"La clase {name} debe implementar 'train' y 'predict'")
        return super().__new__(cls, name, bases, dct)

In [100]:
# Clase Abstracta Base
class BaseModeloML(metaclass=ModeloMLMeta):
    def train(self, X, y):
        pass
        
    def predict(self, X):
        pass

In [101]:
# Implementación de un Modelo de Regresión Lineal
class ModeloRegresion(BaseModeloML):
    def __init__(self):
        self.model = LinearRegression()

    @medir_tiempo  # Decorador para medir tiempo
    def train(self, X, y):
        self.model.fit(X, y)

    @medir_tiempo  # Decorador para medir tiempo
    def predict(self, X):
        return self.model.predict(X)

In [102]:
# Generar datos sintéticos
np.random.seed(42)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [103]:
X[0:10]

array([[0.74908024],
       [1.90142861],
       [1.46398788],
       [1.19731697],
       [0.31203728],
       [0.31198904],
       [0.11616722],
       [1.73235229],
       [1.20223002],
       [1.41614516]])

In [104]:
y[0:10]

array([[6.33428778],
       [9.40527849],
       [8.48372443],
       [5.60438199],
       [4.71643995],
       [5.29307969],
       [5.82639572],
       [8.67878666],
       [6.79819647],
       [7.74667842]])

In [105]:
# Prueba con Decorador (Mide Tiempo de Ejecución)
modelo_con_decorador = ModeloRegresion()
modelo_con_decorador.train(X, y)
y_pred = modelo_con_decorador.predict(X)

train ejecutado en 0.000988 segundos
predict ejecutado en 0.000999 segundos


In [106]:
# Prueba sin Decorador (Medir Manualmente el Tiempo)
modelo_sin_decorador = LinearRegression()

In [107]:
inicio = time.time()
modelo_sin_decorador.fit(X, y)
fin = time.time()
print(f"Entrenamiento sin decorador ejecutado en {fin - inicio:.6f} segundos")

Entrenamiento sin decorador ejecutado en 0.001003 segundos


In [108]:
inicio = time.time()
y_pred_sin = modelo_sin_decorador.predict(X)
fin = time.time()
print(f"Predicción sin decorador ejecutado en {fin - inicio:.6f} segundos")

Predicción sin decorador ejecutado en 0.001005 segundos


In [109]:
# Evaluación del Modelo con decorador
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

In [110]:
print(f"\nMétricas del modelo con decorador:")
print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Coeficiente de determinación (R²): {r2:.2f}")


Métricas del modelo con decorador:
Error cuadrático medio (MSE): 0.81
Coeficiente de determinación (R²): 0.77


In [111]:
# Evaluación del Modelo sin decorador
mse = mean_squared_error(y, y_pred_sin)
r2 = r2_score(y, y_pred_sin)

In [112]:
print(f"\nMétricas del modelo sin decorador:")
print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Coeficiente de determinación (R²): {r2:.2f}")


Métricas del modelo sin decorador:
Error cuadrático medio (MSE): 0.81
Coeficiente de determinación (R²): 0.77


In [113]:
# Inspección con inspect
print("\n Métodos del modelo inspeccionados dinámicamente:")
for nombre, metodo in inspect.getmembers(ModeloRegresion, predicate=inspect.isfunction):
    print(f"  - {nombre}{inspect.signature(metodo)}")


 Métodos del modelo inspeccionados dinámicamente:
  - __init__(self)
  - predict(*args, **kwargs)
  - train(*args, **kwargs)


rendimiento en tabla de rendimiento

Mantenibilidad


In [114]:
# Codigo sin decorador

In [115]:
class ModeloRegresion:
    def __init__(self):
        self.model = LinearRegression()

    def train(self, X, y):
        inicio = time.time()
        self.model.fit(X, y)
        fin = time.time()
        print(f"train ejecutado en {fin - inicio:.6f} segundos")

    def predict(self, X):
        inicio = time.time()
        resultado = self.model.predict(X)
        fin = time.time()
        print(f"predict ejecutado en {fin - inicio:.6f} segundos")
        return resultado


In [116]:
class ModeloRegresion(BaseModeloML):
    def __init__(self):
        self.model = LinearRegression()

    @medir_tiempo
    def train(self, X, y):
        self.model.fit(X, y)

    @medir_tiempo
    def predict(self, X):
        return self.model.predict(X)


3 Complejidad   Se puede aplicar a varias clases ya que existen muchos modelos que se pueden implementar de esta manera.
Por ejemplo un modelo de arbol de decisión:

In [117]:
class ModeloArbolDecision(BaseModeloML):
    def __init__(self):
        self.model = DecisionTreeRegressor()

    @medir_tiempo
    def train(self, X, y):
        self.model.fit(X, y)

    @medir_tiempo
    def predict(self, X):
        return self.model.predict(X)


In [118]:
# Prueba con Decorador (Mide Tiempo de Ejecución)
modelo_arbol_con_decorador = ModeloArbolDecision()
modelo_arbol_con_decorador.train(X, y)
y_pred_arbol = modelo_arbol_con_decorador.predict(X)

train ejecutado en 0.002003 segundos
predict ejecutado en 0.000000 segundos
